In [1]:
from anndata import AnnData
import pandas as pd
import numpy as np
import scipy as sp
import typing
from datetime import datetime
import cnmfsns as cn

obj = cn.Dataset.from_h5ad("../../../testdir/diamandis.h5ad")
obj.adata

AnnData object with n_obs × n_vars = 77 × 3560
    obs: 'patient_id', 'simple_category', 'detailed_category', 'age', 'sex', 'location of resection', 'ATRX', 'p53', 'MGMT'
    uns: 'cnmfsns_version', 'history', 'is_normalized'

In [2]:
obj = cn.Dataset.from_h5ad("/mnt/c/Users/verhe/MorrissyLab Dropbox/Proteomics/tmt_proteomics/cNMF/cnmf_results/diamandis_1_kl.h5ad",
                           force_migrate=True)

/home/tbverhey/github/cNMF-SNS/src/cnmfsns/io.py:58: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  new_adata = ad.AnnData(X=raw, obs=adata.obs, var=adata.var, uns=adata.uns)


In [3]:
obj.to_df(normalized=False)

,A1BG,A2M,AAAS,AAK1,AARS1,AARS2,AARSD1,AASDHPPT,AASS,ABAT,...,ZFR,ZMPSTE24,ZNF207,ZNF326,ZNF428,ZNF512,ZPR1,ZRANB2,ZW10,ZYX
sample_1,1.217576,2.277175,2237880.25,0.847747,130885000.0,3426300.0,2889550.0,7239700.0,2.271401,2.222923,...,1.242628,3775550.0,8502300.0,1935900.0,2289250.000,1.508712,2.957420,2725350.0,2976800.000,1.463501
sample_2,1.936555,2.039212,1864476.50,0.744807,138950000.0,2413700.0,2070000.0,10553000.0,0.647617,2.519262,...,7.708803,3576000.0,7918700.0,6395950.0,3651500.000,1.806712,1.718108,2989000.0,2392150.000,1.610209
sample_3,1.315138,1.168427,1391219.25,0.589671,154024992.0,2262500.0,3211100.0,7216350.0,1.958364,2.923343,...,1.577428,4611050.0,6635700.0,5227650.0,4863525.000,2.277672,7.807588,2773100.0,2419750.000,0.858923
sample_4,0.602541,0.998360,1343487.50,0.876915,117023504.0,1914700.0,4512400.0,13270500.0,0.633954,2.135168,...,3.055330,2950000.0,11180750.0,7869050.0,5014500.000,4.523919,2.382620,4902250.0,3549050.000,1.573727
sample_5,0.608415,1.874264,2133900.00,0.617222,97027504.0,1573700.0,3186750.0,5737350.0,1.971140,0.938534,...,2.810862,10586800.0,9753300.0,9526950.0,2129500.000,1.347608,2.936205,4727000.0,2599700.000,4.023251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sample_82,1.047198,1.286186,2493479.75,0.505767,83618000.0,1984800.0,4793400.0,11909350.0,0.805266,0.677109,...,0.000000,0.0,4422000.0,5202200.0,3650800.000,1.198039,1.559713,3535300.0,2042300.000,0.198441
sample_83,1.429844,1.740493,1619150.00,0.164952,103500496.0,2711400.0,4801700.0,8096300.0,1.816363,1.420873,...,4.733751,8726400.0,10771100.0,11053350.0,3082389.750,2.438255,1.362427,0.0,3388600.000,1.112803
sample_84,0.525039,1.157046,2277400.00,0.234177,123025000.0,2240800.0,4187250.0,11874000.0,0.695517,2.074553,...,1.505020,4824400.0,9288200.0,10197050.0,3387800.000,2.661393,2.455899,3491250.0,3164200.000,1.066194
sample_85,1.897004,4.228630,2557500.00,0.464298,91074000.0,2768100.0,1396100.0,7779950.0,0.977587,1.158232,...,0.926466,6231200.0,4108600.0,3036400.0,1886238.500,1.247943,1.183615,1858500.0,919857.875,1.429665
